In [19]:
import torch
import numpy as np

In [20]:
a = torch.Tensor(3)
print(a)

tensor([0., 0., 0.])


In [21]:
b = torch.Tensor([1,2])
print(b)

tensor([1., 2.])


In [22]:
c = torch.Tensor([2,2])
print(c)

tensor([2., 2.])


In [23]:
d = torch.rand([2,2,2])
print(d)

tensor([[[0.2219, 0.3330],
         [0.0015, 0.0561]],

        [[0.7023, 0.8941],
         [0.7693, 0.2842]]])


In [24]:
x = torch.randn([3,5])
y = torch.randn([5,4])
z = x @ y
print(z.shape)

torch.Size([3, 4])


In [25]:
z = z+z 

In [26]:
print(z.numpy())
print(type(z)) # torch tensor
print(type(z.numpy())) # numpy array)

[[  1.9752926    5.3772435   -0.19811274   9.7257805 ]
 [  2.930745     2.6033418    1.2908676    1.8914489 ]
 [  5.185331   -11.353455    -1.0462513    3.7863462 ]]
<class 'torch.Tensor'>
<class 'numpy.ndarray'>


In [30]:
x = torch.Tensor(np.random.normal([3,5]))
print(type(x)) # torch tensor

<class 'torch.Tensor'>


# Autodiff

In [34]:
x = torch.tensor(1.0, requires_grad=True)



def u(x):
    return x*x
def g(u):
    return -u

In [39]:
dgdx = torch.autograd.grad(g(u(x)), x)[0]

print(dgdx)

tensor(-2.)


In [69]:
x = torch.tensor(([1,2,3],[4,5,6]))
print(x)



tensor([[1, 2, 3],
        [4, 5, 6]])


In [70]:
x_stacked_0 = torch.stack([x,x],0)
print(x_stacked_0)
print(x_stacked_0.size())

tensor([[[1, 2, 3],
         [4, 5, 6]],

        [[1, 2, 3],
         [4, 5, 6]]])
torch.Size([2, 2, 3])


In [71]:

x_stacked_1 = torch.stack([x,x],1)
print(x_stacked_1)
print(x_stacked_1.size())

tensor([[[1, 2, 3],
         [1, 2, 3]],

        [[4, 5, 6],
         [4, 5, 6]]])
torch.Size([2, 2, 3])


In [73]:

x_stacked_2 = torch.stack([x,x],2)
print(x_stacked_2)
print(x_stacked_2.size())

tensor([[[1, 1],
         [2, 2],
         [3, 3]],

        [[4, 4],
         [5, 5],
         [6, 6]]])
torch.Size([2, 3, 2])


In [89]:
print(x*x)
print(x)
print(torch.ones_like(x))

tensor([[[0.1353, 0.0530, 0.0104, 1.9360, 0.6952]],

        [[0.1051, 0.0828, 1.3930, 0.0761, 2.3193]],

        [[0.4783, 0.0218, 1.4749, 2.7523, 1.4608]]])
tensor([[[ 0.3678,  0.2302, -0.1019,  1.3914,  0.8338]],

        [[ 0.3241, -0.2877,  1.1803,  0.2758,  1.5229]],

        [[ 0.6916,  0.1477, -1.2145,  1.6590,  1.2086]]])
tensor([[[1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.]]])


In [91]:

x_stacked_square = torch.stack([x*x,x, torch.ones_like(x) ],-1)

print(x_stacked_square)
print(x_stacked_square.size())


tensor([[[[ 0.1353,  0.3678,  1.0000],
          [ 0.0530,  0.2302,  1.0000],
          [ 0.0104, -0.1019,  1.0000],
          [ 1.9360,  1.3914,  1.0000],
          [ 0.6952,  0.8338,  1.0000]]],


        [[[ 0.1051,  0.3241,  1.0000],
          [ 0.0828, -0.2877,  1.0000],
          [ 1.3930,  1.1803,  1.0000],
          [ 0.0761,  0.2758,  1.0000],
          [ 2.3193,  1.5229,  1.0000]]],


        [[[ 0.4783,  0.6916,  1.0000],
          [ 0.0218,  0.1477,  1.0000],
          [ 1.4749, -1.2145,  1.0000],
          [ 2.7523,  1.6590,  1.0000],
          [ 1.4608,  1.2086,  1.0000]]]])
torch.Size([3, 1, 5, 3])


In [86]:
w = torch.tensor(torch.randn([3,1]), requires_grad=True)

opt = torch.optim.Adam([w],lr=0.1)


def model(x):
    f = torch.stack([x * x, x, torch.ones_like(x)], 1)
    yhat = torch.squeeze(f @ w, 1)
    return yhat


def compute_loss(y, yhat):
    loss = torch.nn.functional.mse_loss(yhat,y)
    return loss


def generate_data():
    x = torch.rand(100)*20 -10
    y = 5*x*x + 3
    return x,y

def train_step():
    x, y = generate_data()
    
    yhat = model(x)
    loss = compute_loss(y, yhat)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
for _ in range(1000):
    train_step()
        
print(w.detach().numpy())

[[4.9922624e+00]
 [6.8596704e-04]
 [3.4647186e+00]]


/var/folders/f2/gq8xgm9x491cqwtct_tgpy0h0000gn/T/ipykernel_23390/2998677527.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w = torch.tensor(torch.randn([3,1]), requires_grad=True)


# Encapsulate the model with Modules



In [92]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.rand(1))
        self.b = nn.Parameter(torch.rand(1))
        
    def forward(self,x):
        yhat = self.a*x + self.b
        return yhat

In [96]:
x = torch.arange(100,dtype=torch.float32)
net = Net()
y = net(x)


In [97]:
for p in net.parameters():
    print(p)

Parameter containing:
tensor([0.2963], requires_grad=True)
Parameter containing:
tensor([0.1923], requires_grad=True)


In [98]:
x = torch.arange(100,dtype=torch.float32) /100
y = 5*x +3 + torch.rand(100) * 0.3

In [100]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)


for i in range(10000):
    net.zero_grad()
    yhat = net(x)
    loss = criterion(yhat,y)
    loss.backward()
    optimizer.step()
    
print(net.a,net.b)

Parameter containing:
tensor([5.0385], requires_grad=True) Parameter containing:
tensor([3.1362], requires_grad=True)


torch.nn.Linear is a predifined module following the one that we set above

In [101]:
class Net(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(1, 1)

    def forward(self, x):
        yhat = self.linear(x.unsqueeze(1)).squeeze(1)
        return yhat

In [102]:
net = Net()
for p in net.parameters():
    print(p)

Parameter containing:
tensor([[0.5225]], requires_grad=True)
Parameter containing:
tensor([0.2610], requires_grad=True)


In [103]:
model = nn.Sequential(
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,10),
)

# Broadcasting